# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [143]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [144]:
orders = pd.read_csv("./Orders.csv")
orders.shape # (397924, 14)
orders.columns # ['Unnamed: 0', 'InvoiceNo', 'StockCode', 'year', 'month', 'day', 'hour', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'amount_spent']
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [145]:
# group by customer
#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?
amount_spent_by_customer = orders.groupby(["CustomerID"]).agg({"amount_spent": "sum"})
amount_spent_by_customer

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [146]:
#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?
# amount_spent_by_customer.describe() # this only gives me the 75% percentile, not the 75% to 95% and the last 5%
amount_spent_by_customer["customer_type"] = pd.qcut(amount_spent_by_customer["amount_spent"], [0, 0.75, 0.95, 1], ["Regular", "Preferred Customers", "VIP"])
amount_spent_by_customer = amount_spent_by_customer.rename(columns={"amount_spent": "total_customer_spent_amount"}) # renaming so it doesn't conflict with amount_spent in original
amount_spent_by_customer

,total_customer_spent_amount,customer_type
CustomerID,,
12346,77183.60,VIP
12347,4310.00,Preferred Customers
12348,1797.24,Preferred Customers
12349,1757.55,Preferred Customers
12350,334.40,Regular
...,...,...
18280,180.60,Regular
18281,80.82,Regular
18282,178.05,Regular


In [147]:
orders_with_customer_type = pd.merge(left=orders, right=amount_spent_by_customer, left_on="CustomerID", right_on="CustomerID", how="inner")
# orders_with_customer_type.shape # (397924, 15) Same as original, no issues
orders_with_customer_type.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,total_customer_spent_amount,customer_type
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21,Preferred Customers
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred Customers
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21,Preferred Customers
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred Customers
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred Customers


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [148]:
only_VIP = orders_with_customer_type[orders_with_customer_type["customer_type"] == "VIP"]
only_VIP["CustomerID"].nunique() # There are only 217 VIP customers

217

In [149]:
only_VIP.groupby(["Country"]).agg({"customer_type": "nunique", "CustomerID": "nunique"}).sort_values(by="CustomerID", ascending=False)
# researched .sort_values to improve visibility
# the nunique allows us to find unique customers, instead of amount of purchases

# NOTE: RESULT: UK has the highest amount of VIP Customers. 177.

# NOTE: INTERESTING_NOTE: The sum of VIP unique customers with this groupby adds to 218, one more than the total 217 VIP customers, this could be due to a customer purchasing in other countries. However the difference is very small and I need to work on the project :/

,customer_type,CustomerID
Country,,
United Kingdom,1,177
Germany,1,10
France,1,9
Switzerland,1,3
Spain,1,2
Portugal,1,2
Australia,1,2
EIRE,1,2
Japan,1,2


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [150]:
preferred_and_VIP = orders_with_customer_type[(orders_with_customer_type["customer_type"] == "VIP") | (orders_with_customer_type["customer_type"] == "Preferred Customers")]
preferred_and_VIP["CustomerID"].nunique() # There are only 1085 VIP customers

1085

In [ ]:
preferred_and_VIP.groupby(["Country"]).agg({"customer_type": "nunique", "CustomerID": "nunique"}).sort_values(by="CustomerID", ascending=False)
# researched .sort_values to improve visibility
# the nunique allows us to find unique customers, instead of amount of purchases

# RESULT: UK has the highest amount of VIP Customers. 932.

# NOTE: INTERESTING_NOTE: Same as in Q2, The sum of VIP/Preferred unique customers with this groupby adds to 1091, siz more than the total 1085 VIP customers, this could be due to a customer purchasing in other countries. However the difference is very small and I need to work on the project :/

## removed the csv to prevent issues with gihub uploading big files.

,customer_type,CustomerID
Country,,
United Kingdom,2,932
Germany,2,39
France,2,29
Belgium,2,12
Switzerland,2,9
Spain,2,9
Portugal,2,7
Norway,2,7
Italy,1,5
